In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('merged_data.csv')
X = df.drop(columns=['Trafikkmengde'])
y = df['Trafikkmengde']
X_train, X_test, y_train, y_test = train_test_split(X, y ,shuffle=False, test_size=0.2)

In [17]:
X_train.fillna(inplace=True, method='ffill')
X_train.isna().sum()

Solskinstid       0
Lufttemperatur    0
Vindstyrke        0
Lufttrykk         0
Vindkast          0
Globalstraling    0
Vindretning       0
Ukedag            0
Maaned            0
Aarstall          0
Klokkeslett       0
Rod_dag           0
dtype: int64

In [4]:
y_test

65353   NaN
65354   NaN
65355   NaN
65356   NaN
65357   NaN
         ..
69691   NaN
69692   NaN
69693   NaN
69694   NaN
69695   NaN
Name: Trafikkmengde, Length: 4343, dtype: float64

In [18]:
my_dict = {'a': 10, 'b': 30, 'c': 5, 'd': 50}

key_with_max_value = max(my_dict, key=my_dict.get)
print(key_with_max_value)


d
